In [1]:
# !which python
# !pip install requests beautifulsoup4 pandas

In [1]:
import requests, time 
# import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from LIB.UTILS.log import get_logger

In [2]:
 get_logger

SyntaxError: invalid character in identifier (<ipython-input-2-17b34830e8e0>, line 1)

In [3]:
log = get_logger('poc') 

SyntaxError: invalid character in identifier (<ipython-input-3-c87e769a9152>, line 1)